# Desafio Movile

Neste desafio, estudamos o problema de classificação de spams em mensagens de SMS. 

Foram fornecidas pela Wavy amostras de mensagens de diferentes operadoras.

In [357]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
pd.set_option('display.max_colwidth', -1)
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer


In [358]:
dataset = pd.read_csv('SPAM Data _ Akari - SPAM Data.csv')
original_dataset = dataset.copy()

In [359]:
dataset.head(10)

,vendor,mensagem,destino,spam,total
0,superible,"A OI esta com 1 oferta Especial para sua nova Empresa adquira hj mesmo telefone Fixo + Banda Larga, ligue 0800 942 5512 ou Responda ok q entraremos em contato",OI,False,1667
1,superible,"A OI esta com uma oferta Especial para sua nova Empresa adquira hoje mesmo Fixo + Banda Larga, ligue 0800 942 5512 ou Responda ok que entraremos em contato.",OI,False,2130
2,mailserr,"Parabens, seu CNPJ esta ativo! A Oi tem OFERTA EXCLUSIVA em voz e internet para sua empresa. Responda OK que ligamos para voce ou ligue 0800 291 2253",OI,False,909
3,zootude,1/2 Para voce receber as nossas ofertas adicione nosso numero 22992872243 em seus contatos e nos mande um oi. De 24 a 26 SUPER SEMANA DA LIMPEZA NO SUPERMERCADO,NaN,False,241
4,zootude,2/2 a CLARO! Ja deixei todo mundo avisado por aqui para soltar a promocao para esse codigo! Corre que a oferta vai ate as 20:00hs. Vem para NET!,CLARO,False,3863
5,zootude,2/2 a CLARO! Ja deixei todo mundo avisado por aqui para soltar a promocao para esse codigo! Corre que a oferta vai ate as 20:00hs. Vem para NET!,CLARO,False,101
6,zootude,"A claro tem oferta imperdivel para voce 3,5GB internet ligacoes ilimitadas p qualquer operadora apenas R$42,99 Venha a Claro em Tres Rios, altere seu plano hoje",CLARO,False,1311
7,zootude,"A claro tem oferta imperdivel para voce 3,5GB internet ligacoes locais ilimitadas por R$42,99 Venha a loja Claro no Shopping Estacao Itaipava e altere seu plano",CLARO,False,797
8,zootude,"A claro tem oferta imperdivel para voce 3,5GB internet ligacoes locais ilimitadas R$42,99 Venha a Claro em Barra do Pirai R Gov Portela e altere seu plano hoje",CLARO,False,932
9,zootude,"A claro tem oferta imperdivel para voce, 3,5GB internet ligacoes ilimitadas p qualquer operadora apenas R$42,99 Venha a Claro em Teresopolis e altere seu plano",CLARO,False,1895


Na amostra acima, podemos ver que temos 5 features:
* Vendor: Nome da empresa que enviou a mensagem
* Mensagem: texto da mensagem
* Destino: Operadora do destinatário da mensagem
* Spam: se foi classificada como spam (true) ou não (false)
* Total: Quantidade de cópias da mensagem enviadas

## Abordagens

Existem inúmeras formas de explorar o problema. Primeiro, devemos olhar bem as características dos nossos dados.

In [360]:
dataset['destino'].value_counts()

OI        74
TIM       50
CLARO     46
VIVO      9 
NEXTEL    8 
Oi        6 
Name: destino, dtype: int64

In [361]:
dataset['spam'].value_counts()

False    991
True     6  
Name: spam, dtype: int64

Aqui temos um problema em que uma classe (negativa) é muito mais predominante do que a outra (positiva). Casos assim não são triviais de serem solucionados. Vamos então primeiramente explorar os dados e analisar uma possível solução.

Podemos começar vendo o que temos nesses SMSs classificados como spam

In [362]:
dataset.loc[dataset['spam'] == True]

,vendor,mensagem,destino,spam,total
772,mailserr,OI. Temos uma OFERTA especial de CELULAR ILIMITADO para todo BRASIL + 10GB de INTERNET. responda OK que retornarmos para voce ou ligue 0800 291 2253,NaN,True,2276
783,quasiyo,"Ola, somos da TIM! Parabens! Seu chip esta ativado, realize uma ligacao de 30 segundos para confirmar o funcionamento. Digite se ja esta utilizando, 2 se nao.",NaN,True,495
785,quasiyo,"Ola, somos da TIM! Seu chip foi ativado e liberado para fazer ligacao. Utilize com urgencia, p confirmar o sinal! Digite 1 se ja esta utilizando, 2 se nao.",NaN,True,1041
786,quasiyo,"Ola, somos da TIM! Seu chip foi ativado e liberado para fazer ligacao. Utilize com urgencia, p confirmar o sinal! Digite 1 se ja esta utilizando, 2 se nao.",NaN,True,458
787,quasiyo,"Ola, somos da TIM! Seu chip ja foi ativado e esta gerando fatura. Digite 1 se ja realizou alguma ligacao com seu chip novo, 2 se nao.",NaN,True,283
788,quasiyo,"Ola, somos da TIM! Verificamos que voce ainda nao utilizou seu chip, e estamos gerando fatura! Faca uma ligacao com urgencia usando o seu chip da TIM.",NaN,True,193


Uma possível solução seria propor uma aumentação de dados. Como uma mensagem de propaganda de uma operadora enviada para outra operadora é considerado um spam, podemos usar isso como estratégia para aumentar os dados:

Primeiro podemos criar cópias de ofertas da Oi para outras operadoras e colocá-las como exemplos de spams

In [363]:
dataset.loc[(original_dataset['destino'] == 'CLARO')]

,vendor,mensagem,destino,spam,total
4,zootude,2/2 a CLARO! Ja deixei todo mundo avisado por aqui para soltar a promocao para esse codigo! Corre que a oferta vai ate as 20:00hs. Vem para NET!,CLARO,False,3863
5,zootude,2/2 a CLARO! Ja deixei todo mundo avisado por aqui para soltar a promocao para esse codigo! Corre que a oferta vai ate as 20:00hs. Vem para NET!,CLARO,False,101
6,zootude,"A claro tem oferta imperdivel para voce 3,5GB internet ligacoes ilimitadas p qualquer operadora apenas R$42,99 Venha a Claro em Tres Rios, altere seu plano hoje",CLARO,False,1311
7,zootude,"A claro tem oferta imperdivel para voce 3,5GB internet ligacoes locais ilimitadas por R$42,99 Venha a loja Claro no Shopping Estacao Itaipava e altere seu plano",CLARO,False,797
8,zootude,"A claro tem oferta imperdivel para voce 3,5GB internet ligacoes locais ilimitadas R$42,99 Venha a Claro em Barra do Pirai R Gov Portela e altere seu plano hoje",CLARO,False,932
9,zootude,"A claro tem oferta imperdivel para voce, 3,5GB internet ligacoes ilimitadas p qualquer operadora apenas R$42,99 Venha a Claro em Teresopolis e altere seu plano",CLARO,False,1895
10,centigen,"A Claro tem planos para sua empresa economizar! 5GB + Ligacoes ILIMITADAS por R$72,30/mes! Responda sim e receba um contato AGORA!",CLARO,False,770
30,zootude,"ADRIANA, a Claro tem uma oferta para voce. Ligue agora 1052 e descubra.",CLARO,False,198
43,zootude,"ALEXANDRE, a Claro tem uma oferta para voce. Ligue agora 1052 e descubra.",CLARO,False,131
139,difize,"CLARO CONTROLE Traga sua linha e tenha o 4G mais rapido do brasil + Ligacoes ILIMITADAS e GANHE uma linha claro fixo apenas 72,80 responda SIM OU 0800 042 0177",CLARO,False,15841


Podemos copiar algumas mensagens para outras operadoras:

In [364]:
def cria_spam(aumentacao, destino):
    global dataset
    aumentacao['spam'] = True

    if destino != 'OI':
        aumentacao['destino'] = 'OI'
        dataset = dataset.append(aumentacao)   
        
    if destino != 'CLARO':
        aumentacao['destino'] = 'CLARO'
        dataset = dataset.append(aumentacao)
    
    if destino != 'TIM':
        aumentacao['destino'] = 'TIM'
        dataset = dataset.append(aumentacao)
        
    if destino != 'VIVO':
        aumentacao['destino'] = 'VIVO'
        dataset = dataset.append(aumentacao)
        
    if destino != np.nan:
        aumentacao['destino'] = np.nan
        dataset = dataset.append(aumentacao)
    

In [365]:
dataset = original_dataset.copy()

aumentacao = original_dataset.loc[(original_dataset['destino'] == 'OI') & (original_dataset['mensagem'].str.contains('sua nova Empresa') | original_dataset['mensagem'].str.contains('Responda SIM'))]
cria_spam(aumentacao, 'OI')

aumentacao = original_dataset.loc[(original_dataset['destino'] == 'TIM') & (original_dataset['mensagem'].str.contains('condicoes') | original_dataset['mensagem'].str.contains('responda Sim') | original_dataset['mensagem'].str.contains('RESP: SIM') | original_dataset['mensagem'].str.contains('digite SIM'))]
cria_spam(aumentacao, 'TIM')

aumentacao = original_dataset.loc[(original_dataset['destino'] == 'VIVO') & (original_dataset['mensagem'].str.contains('Venha') | original_dataset['mensagem'].str.contains('Responda'))]
cria_spam(aumentacao, 'VIVO')

aumentacao = original_dataset.loc[(original_dataset['destino'] == 'CLARO') & (original_dataset['mensagem'].str.contains('GRATIS') |original_dataset['mensagem'].str.contains('oferta') | original_dataset['mensagem'].str.contains('Responda'))]
cria_spam(aumentacao, 'CLARO')

In [366]:
dataset['spam'].value_counts()

False    991
True     378
Name: spam, dtype: int64

Agora que fizemos a aumentação de dados, podemos transformar as mensagens em vetores usando o TfidfVectorizer do sklearn. Notei também que não há relação com a empresa que enviou a mensagem, nem o número de cópias da mensagem, podemos então descartar essas features.

In [457]:
label = dataset['spam']
label = np.array(label)
data = dataset.drop(['spam', 'vendor', 'total'], axis=1)
data = data.fillna('TODAS')
data = np.array(data)


ops = {'OI': 0, 'VIVO': 1, 'TIM': 2, 'CLARO': 3, 'TODAS': 4, 'NEXTEL': 5, 'Oi':0}

operadora = np.zeros((len(data), 6))
for i in range(len(data)):
    operadora[i][ops[data[i][1]]] = 1

data = np.column_stack((data[:,0], operadora))

print(data.shape)

tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words=['e', 'ou','a','o','para','de','da','do','na','em','no'],ngram_range=(1,1))
data[:,0] = tfidf.fit_transform(data[:,0])

#Separar em treino e teste aleatoriamente
X_train, X_test, y_train, y_test =  train_test_split(data, label, test_size=0.15, random_state=5)

#Separar em treino e validação
X_train, X_valid, y_train, y_valid =  train_test_split(X_train, y_train, test_size=0.3, random_state=5)


print(y_train.shape)
print(X_train.shape)

(1369, 7)
(814,)
(814, 7)


Agora que temos os dados preparados, podemos aplicar algum método para definir se os SMS são ou não spams. Iremos aplicar aqui uma simples regressão linear para reduzir tempo e custo computacional.

In [458]:
from sklearn.linear_model import LinearRegression
regressao = LinearRegression().fit(X_train, y_train)
# regressao.score(X_valid, y_valid)

ValueError: setting an array element with a sequence.

In [459]:
y_train

array([False, False, False, False, False, False, False, False,  True,
       False,  True, False,  True,  True,  True,  True, False, False,
        True, False, False, False, False, False, False, False, False,
        True, False,  True, False, False, False, False, False, False,
       False,  True, False,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True, False, False, False, False, False,
       False, False, False, False,  True, False, False,  True, False,
       False, False, False, False,  True,  True, False,  True, False,
       False,  True,  True, False, False, False, False, False,  True,
       False, False,  True,  True, False, False, False,  True,  True,
        True, False, False, False, False,  True, False, False, False,
        True, False, False, False,  True, False,  True,  True, False,
       False, False, False,  True,  True, False,  True, False, False,
        True, False, False, False, False, False,  True, False, False,
       False, False,

In [460]:
y_train.shape

(814,)

In [468]:
x = y_train.reshape((814,1)).transpose()

In [469]:
x

array([[False, False, False, False, False, False, False, False,  True,
        False,  True, False,  True,  True,  True,  True, False, False,
         True, False, False, False, False, False, False, False, False,
         True, False,  True, False, False, False, False, False, False,
        False,  True, False,  True, False,  True,  True,  True,  True,
        False,  True,  True,  True, False, False, False, False, False,
        False, False, False, False,  True, False, False,  True, False,
        False, False, False, False,  True,  True, False,  True, False,
        False,  True,  True, False, False, False, False, False,  True,
        False, False,  True,  True, False, False, False,  True,  True,
         True, False, False, False, False,  True, False, False, False,
         True, False, False, False,  True, False,  True,  True, False,
        False, False, False,  True,  True, False,  True, False, False,
         True, False, False, False, False, False,  True, False, False,
      